In [211]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import graph_objects as go

# Analysis of Simulation Benchmarking Experiments

The benchmarking experiments were run on  a Dell XPS-15. The metrics reported in the paper come from these experiments.

In [212]:
cols={0:"time_string",1:"time_taken_lec",2:"time_taken_safety_controller",
      3:"total_time",4:"moet",5:"mean_et",6:"avg_iterations",7:"obstacle_presence"}
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0","1.5"]
runtimes = ['10','25']
file_str = "benchmark_experiments_{}_track_porto_{}_{}.csv"
dfs = []
count = 1
for alg in algorithms:
    for speed in speeds:
        for rt in runtimes:
            file_path = file_str.format(alg,speed,rt)
            df = pd.read_csv(file_path,header=None).rename(columns=cols)
            df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
            df=df.drop(columns=["time_taken_lec","total_time","time_taken_safety_controller"])
            df["sc_usage"] = 1 - df["ml_usage"]
            df["ml_usage"]=df["ml_usage"]*100
            df["sc_usage"]=df["sc_usage"]*100
            df["ml_controller"] = alg.replace("_rtreach","")
            df["speed"] = speed
            df['runtime'] = rt
            dfs.append(df)
            

In [213]:
def replace_e2e(row):
    if(row=="e2e_image"):
        return "IL (Camera)"
    elif(row=="e2e"):
        return "IL (Lidar)"
    else:
        return row.upper()
def replace_obs(row):
    if(row==0):
        return "No" 
    else: 
        return "Yes"

In [238]:
all_dfs = pd.concat(dfs)
all_dfs["ml_controller"]=all_dfs["ml_controller"].apply(replace_e2e)
all_dfs["obstacle_presence"] = all_dfs["obstacle_presence"].apply(replace_obs)
all_dfs['moet'] = all_dfs['moet'] *1000
all_dfs['mean_et'] = all_dfs['mean_et'] * 1000
all_dfs = all_dfs.reset_index().drop(columns='index')
all_dfs

,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage,ml_controller,speed,runtime
0,01/04/2021/17:06:07,8.478,6.64321,3.96654,No,94.933493,5.066507,IL (Camera),0.5,10
1,01/04/2021/17:07:36,8.315,6.60067,3.97587,No,88.899489,11.100511,IL (Camera),0.5,10
2,01/04/2021/17:09:06,8.507,6.43541,3.98251,No,94.946925,5.053075,IL (Camera),0.5,10
3,01/04/2021/17:10:21,8.885,6.63455,3.96435,No,94.508173,5.491827,IL (Camera),0.5,10
4,01/04/2021/17:11:51,8.353,6.52825,3.98548,No,93.997684,6.002316,IL (Camera),0.5,10
...,...,...,...,...,...,...,...,...,...,...
1577,04/04/2021/11:29:08,31.738,12.50400,8.00000,Yes,21.071717,78.928283,ARS,1.5,25
1578,04/04/2021/11:30:22,26.828,14.13350,5.13871,Yes,30.190451,69.809549,ARS,1.5,25
1579,04/04/2021/11:31:35,27.118,15.29310,5.28830,Yes,34.088801,65.911199,ARS,1.5,25
1580,04/04/2021/11:32:49,26.489,14.33450,5.13442,Yes,33.878201,66.121799,ARS,1.5,25


### Remove Outliers

In [253]:
all_dfs = all_dfs.drop(all_dfs.iloc[list(np.where((all_dfs['moet'])>(3*all_dfs['mean_et']))[0])].index)

In [254]:
all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).count()

time_string  moet  mean_et  \
ml_controller speed obstacle_presence runtime                               
ARS           0.5   No                10                39    39       39   
                                      25                41    41       41   
                    Yes               10                41    41       41   
                                      25                37    37       37   
              1.0   No                10                40    40       40   
                                      25                36    36       36   
                    Yes               10                33    33       33   
                                      25                42    42       42   
              1.5   No                10                44    44       44   
                                      25                44    44       44   
                    Yes               10                36    36       36   
                                      25                43    43       43   
IL (Camera)   0.5   No                10                30    30       30   
                                      25                32    32       32   
                    Yes               10                29    29       29   
                                      25                29    29       29   
              1.0   No                10                30    30       30   
                                      25                30    30       30   
                    Yes               10                28    28       28   
                                      25                29    29       29   
              1.5   No                10                30    30       30   
                                      25                30    30       30   
                    Yes               10                26    26       26   
                                      25                28    28       28   
IL (Lidar)    0.5   No                10                30    30       30   
                                      25                30    30       30   
                    Yes               10                29    29       29   
                                      25                29    29       29   
              1.0   No                10                30    30       30   
                                      25                30    30       30   
                    Yes               10                28    28       28   
                                      25                26    26       26   
              1.5   No                10                30    30       30   
                                      25                30    30       30   
                    Yes               10                25    25       25   
                                      25                27    27       27   
SAC           0.5   No                10                29    29       29   
                                      25                30    30       30   
                    Yes               10                27    27       27   
                                      25                28    28       28   
              1.0   No                10                29    29       29   
                                      25                29    29       29   
                    Yes               10                28    28       28   
                                      25                29    29       29   
              1.5   No                10                26    26       26   
                                      25                28    28       28   
                    Yes               10                25    25       25   
                                      25                28    28       28   

                                               avg_iterations  ml_usage  \
ml_controller speed obstacle_presence runtime                             
ARS           0.5   No                10      

In [255]:
outlier10 = all_dfs[all_dfs['moet']>20]
outlier10 = outlier10[outlier10.runtime=='10']
outlier10.groupby(['ml_controller','runtime','speed']).count()

,,,time_string,moet,mean_et,avg_iterations,obstacle_presence,ml_usage,sc_usage
ml_controller,runtime,speed,,,,,,,


In [256]:
res = round(all_dfs.groupby(["ml_controller","speed","obstacle_presence","runtime"]).agg(["mean","std"]),3)
res

moet        mean_et         \
                                                 mean    std    mean    std   
ml_controller speed obstacle_presence runtime                                 
ARS           0.5   No                10       11.097  2.954   5.991  0.262   
                                      25       27.276  1.592  14.096  0.903   
                    Yes               10       10.039  1.918   6.051  0.248   
                                      25       27.294  1.167  14.285  0.809   
              1.0   No                10       14.233  2.415   5.986  0.228   
                                      25       28.584  2.414  14.101  0.537   
                    Yes               10       11.389  2.551   5.919  0.539   
                                      25       28.260  2.905  14.334  0.821   
              1.5   No                10        9.165  1.480   6.265  0.239   
                                      25       27.248  1.709  14.239  0.336   
                    Yes               10       10.441  2.420   6.209  0.259   
                                      25       27.587  1.638  14.300  0.618   
IL (Camera)   0.5   No                10        8.556  0.251   6.594  0.122   
                                      25       27.218  0.832  14.364  0.694   
                    Yes               10        8.880  0.729   6.422  0.196   
                                      25       27.528  0.973  14.444  0.293   
              1.0   No                10        8.770  0.375   6.615  0.139   
                                      25       27.392  0.672  14.208  0.232   
                    Yes               10        9.308  1.326   6.356  0.618   
                                      25       27.734  1.983  14.444  0.937   
              1.5   No                10        8.922  0.476   6.603  0.100   
                                      25       27.236  0.598  14.376  0.317   
                    Yes               10        9.175  0.552   6.287  0.610   
                                      25       27.400  1.009  14.631  0.834   
IL (Lidar)    0.5   No                10        9.031  0.310   6.620  0.077   
                                      25       28.267  0.958  14.492  0.103   
                    Yes               10        9.312  0.489   6.422  0.233   
                                      25       28.443  1.998  14.694  0.214   
              1.0   No                10        9.133  0.523   6.631  0.051   
                                      25       27.851  0.633  14.574  0.213   
                    Yes               10       10.034  2.096   6.331  0.273   
                                      25       28.591  2.309  14.979  0.702   
              1.5   No                10        9.703  1.263   6.671  0.540   
                                      25       27.876  0.728  15.080  1.264   
                    Yes               10        9.877  1.606   6.462  0.154   
                                      25       28.816  2.167  14.925  0.570   
SAC           0.5   No                10        9.873  2.549   6.397  0.184   
                                      25       27.371  1.182  13.978  0.305   
                    Yes               10        9.006  0.751   6.303  0.159   
                                      25       27.692  2.352  14.063  0.429   
              1.0   No                10        9.738  1.772   6.433  0.236   
                                      25       27.402  2.146  13.755  0.507   
                    Yes               10       10.486  1.695   6.156  0.265   
                                      25       27.644  2.614  13.881  1.064   
              1.5   No                10        9.329  0.833   6.289  0.267   
                                      25       28.124  2.258  13.624  0.382   
                    Yes               10       10.022  1.930   6.301  0.302   
                                      25       27.670  1.790  13.947  0.391   

                   

# Analysis of Hardware Experiments

In [127]:
hw_file_str = "benchmark_hardware_experiments_{}_{}_{}.csv"
algorithms = ["e2e_image_rtreach","e2e_rtreach","sac_rtreach","ars_rtreach"]
speeds = ["0.5","1.0"]
runtimes = ['10','25']
hw_dfs = []
for sp in speeds:
    for alg in algorithms:
        for rt in runtimes:
            file_path = hw_file_str.format(sp,alg,rt)
            try:
                df = pd.read_csv(file_path,header=None).rename(columns=cols)
                df["ml_usage"] = df["time_taken_lec"]/df["total_time"]
                df=df.drop(columns=["time_taken_lec","time_taken_safety_controller","total_time","obstacle_presence"])
                df["sc_usage"] = 1 - df["ml_usage"]
                df["ml_usage"]=df["ml_usage"]*100
                df["sc_usage"]=df["sc_usage"]*100
                df['moet'] = df['moet'] * 1000
                df['mean_et'] = df['mean_et'] * 1000
                df["ml_controller"] = alg.replace("_rtreach","")
                df["speed"] = sp
                df['runtime'] = rt
                hw_dfs.append(df)
            except:
                continue
hw_dfs = pd.concat(hw_dfs)
hw_dfs["ml_controller"]=hw_dfs["ml_controller"].apply(replace_e2e)
hw_dfs

,time_string,moet,mean_et,avg_iterations,ml_usage,sc_usage,ml_controller,speed,runtime
0,03/04/2021/13:25:54,10.935,5.78091,4.99117,17.917072,82.082928,IL (Camera),0.5,10
1,03/04/2021/13:27:15,11.029,6.30026,4.55825,27.399794,72.600206,IL (Camera),0.5,10
2,03/04/2021/13:28:57,13.587,6.11045,4.89243,26.247901,73.752099,IL (Camera),0.5,10
3,03/04/2021/13:30:38,24.368,5.69501,4.36828,20.250907,79.749093,IL (Camera),0.5,10
4,03/04/2021/13:32:13,12.129,5.33000,4.88028,17.869757,82.130243,IL (Camera),0.5,10
...,...,...,...,...,...,...,...,...,...
0,03/04/2021/13:56:42,24.684,11.01770,5.55328,19.607765,80.392235,ARS,1.0,25
1,03/04/2021/13:58:20,23.122,11.38650,5.33877,15.694671,84.305329,ARS,1.0,25
2,03/04/2021/13:59:56,24.557,11.03970,5.27128,29.077913,70.922087,ARS,1.0,25
3,03/04/2021/14:01:38,25.477,11.30330,5.77496,26.734366,73.265634,ARS,1.0,25


In [130]:
res = round(hw_dfs.groupby(["ml_controller","speed","runtime"]).agg(["mean","std"]),3)
res

moet        mean_et        avg_iterations  \
                               mean    std    mean    std           mean   
ml_controller speed runtime                                                
ARS           0.5   10       13.806  6.390   4.883  0.259          4.901   
                    25       23.739  0.638  11.273  0.662          5.949   
              1.0   25       24.530  0.863  11.153  0.178          5.544   
IL (Camera)   0.5   10       13.760  5.314   5.788  0.364          4.835   
                    25       24.486  0.893  11.903  0.381          5.619   
              1.0   25       24.641  1.413  11.147  0.443          5.460   
IL (Lidar)    0.5   10       13.359  2.608   6.221  0.523          4.770   
                    25       24.815  0.989  11.340  0.644          5.366   
              1.0   25       25.025  1.024  10.721  0.293          5.163   
SAC           0.5   10       12.179  1.414   5.288  0.379          4.860   
                    25       24.702  0.923  11.483  0.604          5.835   
              1.0   25       24.420  0.956  10.511  0.463          5.790   

                                   ml_usage        sc_usage         
                               std     mean    std     mean    std  
ml_controller speed runtime                                         
ARS           0.5   10       0.707   20.219  5.297   79.781  5.297  
                    25       0.627   33.939  8.152   66.061  8.152  
              1.0   25       0.238   22.640  5.384   77.360  5.384  
IL (Camera)   0.5   10       0.335   21.013  4.682   78.987  4.682  
                    25       0.327   25.658  5.087   74.342  5.087  
              1.0   25       0.184   17.262  2.109   82.738  2.109  
IL (Lidar)    0.5   10       0.165   21.755  6.905   78.245  6.905  
                    25       0.187   29.149  5.572   70.851  5.572  
              1.0   25       0.066   21.332  2.351   78.668  2.351  
SAC           0.5   10       0.286   16.019  6.917   83.981  6.917  
                    25       0.166   27.618  8.527   72.382  8.527  
              1.0   25       0.807   16.820  3.964   83.180  3.964